In [34]:
import mediapipe as mp
import cv2 as cv
import numpy as np
import time
import sounddevice as sd
import librosa

In [35]:
playbackScreen = cv.imread('playing.png', cv.IMREAD_UNCHANGED) 

In [36]:
class HandDetector:
    INF = 1000

    color_scheme = cv.COLOR_BGR2RGB
    hands = mp.solutions.hands.Hands()
    landmarks = []
    closure_threshold = 0.25

    @staticmethod
    def getLandmarks(frame):
        imgColor = cv.cvtColor(frame, HandDetector.color_scheme)
        processed = HandDetector.hands.process(imgColor)
        result = processed.multi_hand_landmarks
        HandDetector.landmarks = result

    @staticmethod
    def findThumb() -> np.array:
        if HandDetector.landmarks != None:
            thumbLocation = HandDetector.landmarks[0].landmark[mp.solutions.hands.HandLandmark.THUMB_TIP]

            return np.array([thumbLocation.x, thumbLocation.y])
        return np.array([])
    
    @staticmethod
    def findIndex() -> np.array:
        if HandDetector.landmarks != None:
            indexLocation = HandDetector.landmarks[0].landmark[mp.solutions.hands.HandLandmark.INDEX_FINGER_TIP]
            
            return np.array([indexLocation.x, indexLocation.y])
        
        return np.array([])
    

    @staticmethod
    def getMiddleToWristDistance() -> float:
        if HandDetector.landmarks != None:
            middleTipLocation = HandDetector.landmarks[0].landmark[mp.solutions.hands.HandLandmark.MIDDLE_FINGER_TIP]
            wristLocation = HandDetector.landmarks[0].landmark[mp.solutions.hands.HandLandmark.WRIST]

            middleTipCoordinate = np.array([middleTipLocation.x, middleTipLocation.y])
            wristCoordinate = np.array([wristLocation.x, wristLocation.y])

            return np.linalg.norm(middleTipCoordinate - wristCoordinate)
        
        return HandDetector.INF

    
    @staticmethod
    def isClosed() -> bool:
        return HandDetector.getMiddleToWristDistance() <= HandDetector.closure_threshold


In [37]:
sensitivity = 12.32
intensity = 1000

def getShift(dist):
    if dist < 0.045:
        return intensity
    elif 0.045 <= dist <= 0.3:
        return intensity * np.cos(sensitivity * (dist - 0.045))
    else:
        return -1 * intensity

In [38]:
def draw_freqShift(freq, frame):
    position = (10, 25)
    font = cv.FONT_HERSHEY_SIMPLEX
    font_scale = 1

    if freq > 0:
        color = (0, 100, freq * 255)
    else:
        color = (-1 * freq * 255, 100, 0)

    thickness = 1

    cv.putText(frame, str(freq), position, font, font_scale, color, thickness)

def draw_playbackScreen(frame):
    h, w = playbackScreen.shape[:2]
    h = min(h, frame.shape[0])
    w = min(w, frame.shape[1])

    frame[:h, :w] = playbackScreen[:h, :w]

    cv.imshow('cam', frame)

In [ ]:
samplerate = 44100
chunk_duration = 0.5
chunk_size = int(samplerate * chunk_duration)

def record(cam, handClosed):
    audio_chunks = []
    pitch_shifts = []

    def audio_callback(indata, frames, time, status):
        audio_chunks.append(indata.copy())

    stream = sd.InputStream(samplerate=samplerate, channels=1, blocksize=chunk_size, callback=audio_callback)
    stream.start()

    start_time = cv.getTickCount()
    while handClosed:
        ret, frame = cam.read()
        frame = cv.flip(frame, 1)
        HandDetector.getLandmarks(frame)
        
        thumbLoc = HandDetector.findThumb()
        indexLoc = HandDetector.findIndex()

        frequency_shift = getShift(np.linalg.norm(thumbLoc - indexLoc))
        draw_freqShift(int(100 * (frequency_shift / intensity)) / 100, frame)
        pitch_shifts.append(frequency_shift)
        
    
        cv.imshow('cam', frame)
        if cv.waitKey(1) == ord('q'):
            exit()

        handClosed = HandDetector.isClosed()
    
    # playback recording
    stream.stop()

    return audio_chunks, pitch_shifts


volume = 3.0
def playback(cam, audio, pitch_mod):
    processed_audio = [[]]

    min_len = min(len(audio), len(pitch_mod))
    for i in range(min_len):
        ret, frame = cam.read()
        frame = cv.flip(frame, 1)
        draw_playbackScreen(frame)

        chunk = audio[i].flatten()
        shift = pitch_mod[i]

        cv.imshow('cam', frame)
        if cv.waitKey(1) == ord('q'):
            exit()
        

        if shift < 0:
            semitones = -1 * np.log2(-1 * shift)
        elif shift > 0:
            semitones = np.log2(shift) 
        else:
            semitones = 0

        try:
            shifted = librosa.effects.pitch_shift(chunk, sr=samplerate, n_steps=semitones)
            processed_audio.append(shifted)
        except:
            processed_audio.append(chunk)
    
    output = np.concatenate(processed_audio)
    output *= volume
    
    sd.play(output, samplerate)
    sd.wait()

In [ ]:
cam = cv.VideoCapture(0)

while True:
    ret, frame = cam.read()
    frame = cv.flip(frame, 1)
    HandDetector.getLandmarks(frame)

    handClosed = HandDetector.isClosed()
    if handClosed:
        recording, shifts = record(cam, handClosed)
        playback(cam, recording, shifts)

    if not ret:
        break

    if cv.waitKey(1) == ord('q'):
        cam.release()
        cv.destroyAllWindows()
        exit()

    cv.imshow('cam', frame)



KeyboardInterrupt: 